In [63]:
from __future__ import print_function
from collections import defaultdict
import numpy as np
import scipy as sp
import cPickle as pickle
import time
from collections import defaultdict

from util_gradient import *

In [64]:
def init_theta(K, num_users, num_items):
    init_file = "alpha_beta_init_lam0.5_iter50.feature"
#     init_file = "alpha_beta_init_lam1_iter20.feature"
    
    # init_file = "alpha_beta_init.feature"
    alpha, beta_users, beta_items = pickle.load(open(init_file, "rb"))
    # alpha = np.mean(train_rating_array[:, 2])
    # beta_users = np.random.normal(0, 0.5, (num_users, ))
    # beta_items = np.random.normal(0, 0.5, (num_items, ))
    gamma_users = np.random.normal(0, 0.01, (num_users, K))
    gamma_items = np.random.normal(0, 0.01, (num_items, K))
    
    theta_length = (1 + 
                    beta_users.size + 
                    beta_items.size +
                    gamma_users.size + 
                    gamma_items.size)
    
    theta = np.empty((theta_length, ))
    return pack(theta, K, alpha, beta_users, beta_items, gamma_users, gamma_items)

In [65]:
def progress_callback(theta):
    def test_and_get_mse(rating_array, theta, K):
        ratings_predict = [predict_one_rating(user_index, item_index, theta, K) 
                           for user_index, item_index in rating_array[:, :2]]
        ratings = rating_array[:, 2]
        return get_mse(ratings_predict, ratings)
    
    print("train mse:", test_and_get_mse(train_rating_array, theta, K))
    print("valid mse:", test_and_get_mse(valid_rating_array, theta, K))

In [66]:
from scipy.optimize import minimize, fmin_l_bfgs_b

# load preprocessed data
# id <-> index infastructure

(user_id_map_index, 
 user_index_map_id,
 item_id_map_index,
 item_index_map_id) = pickle.load(open("id_index_map.feature", "rb"))
# train_rating_array, valid_rating_array
(train_rating_array, 
 valid_rating_array, 
 all_rating_array) = pickle.load(open("train_valid_all_rating_array.feature", "rb"))

In [67]:
# global variableds
K = 1
lam = 3.0
num_users = len(user_index_map_id)
num_items = len(item_index_map_id)
assert num_users == 35736
assert num_items == 37801

# init theta and grad_buffer
theta = init_theta(K, num_users, num_items)
grad_buffer = np.zeros_like(theta)

In [68]:
# alpha, beta_users, beta_items, gamma_users, gamma_items = unpack(theta, K)
# np.std(gamma_users.reshape((-1,)))

# import math
# def my_round(rating):
#     rating = min(5.0, rating)
#     rating = max(1.0, rating)
#     if rating - math.floor(rating) < 0.05:
#         return math.floor(rating)
#     if math.ceil(rating) - rating < 0.05:
#         return math.ceil(rating)
#     return rating

In [69]:
res = minimize(objective, theta, 
               args       = (grad_buffer, all_rating_array, lam, K), # train_rating_array
               method     = 'L-BFGS-B',
               jac        = gradient,
               options    = {'disp': True, 'maxiter': 5},
               callback   = progress_callback)
theta = res.x

train mse: 0.589261279031
valid mse: 0.594185537913
train mse: 0.590224661
valid mse: 0.595397306115
train mse: 0.592044904359
valid mse: 0.597566338646
train mse: 0.595823594759
valid mse: 0.601585508675
train mse: 0.595360333806
valid mse: 0.601073230635
train mse: 0.595068484163
valid mse: 0.600724123982


In [70]:
# get header_str and user_item_ids to predict
with open('pairs_Rating.txt') as f:
    # read and strip lines
    lines = [l.strip() for l in f.readlines()]
    # stirip out the headers
    header_str = lines.pop(0)
    # get a list of user_item_ids
    user_item_ids = [l.split('-') for l in lines]

# write to output file
f = open('predictions_Rating.txt', 'w')
print(header_str, file=f)
for user_id, item_id in user_item_ids:
    rating = predict_one_rating(user_id_map_index[user_id], 
                                item_id_map_index[item_id], 
                                theta, K)
    rating = min(5.0, rating)
    rating = max(0.0, rating)
    print('%s-%s,%s' % (user_id, item_id, rating), file=f)
f.close()